In [30]:
import os
import random
import sys
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('gb18030') 
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

In [31]:
from summa import keywords
from summa import summarizer
def sortByOrder(text, a, b):
    if text.find(a) == -1:
        _a = a.split(' ')[0]
    else:
        _a = a
    if text.find(b) == -1:
        _b = b.split(' ')[0]
    else:
        _b = b
    ind_a = text.index(_a)
    ind_b = text.index(_b)
    return ind_a < ind_b
def get_keywords_and_sort_with_ratio(text, words_ratio=0.8):
    _keyword = keywords.keywords(text, ratio=words_ratio)
    _keyword = _keyword.split('\n')
    keyword = sorted(_keyword, lambda x,y: -sortByOrder(text, x, y))
    return keyword
def get_keywords_and_sort_with_count(text, words_count=10):
    _keyword = keywords.keywords(text, words=words_count)
    _keyword = _keyword.split('\n')
    keyword = sorted(_keyword, lambda x,y: -sortByOrder(text, x, y))
    return keyword
def _Get_keywords_and_sort_with_ratio(text, words_ratio=1):
    _keyword = keywords.keywords(text, ratio=words_ratio)
    _keyword = _keyword.split('\n')
    _keyword = ' '.join(_keyword)
    _keyword = _keyword.split(' ')
    _keyword = [w if w!='unk' else 'UNK' for w in _keyword ]
    _keyword = [w for w in _keyword if text.find(w) !=-1 ]
    _keyword = sorted(set(_keyword), key = _keyword.index)   #去掉重复元素
    keyword = sorted(_keyword, lambda x,y: -sortByOrder(text, x, y))
    return keyword
def Get_keywords_and_sort_with_ratio(text, words_ratio=1):
    _keyword = keywords.keywords(text, ratio=words_ratio)
    _keyword = _keyword.split('\n')
    _keyword = [w if w!='unk' else 'UNK' for w in _keyword ]
    _keyword = [w for w in _keyword if text.find(w) !=-1 ]
    keyword = sorted(_keyword, lambda x,y: -sortByOrder(text, x, y))
    keyword = ' '.join(keyword)
    keyword = keyword.split(' ')
    keyword = sorted(set(keyword), key = keyword.index)   #去掉重复元素
    return keyword
def Get_keywords_and_sort_with_count(text, words_count=10):
    _keyword = keywords.keywords(text, words=words_count)
    _keyword = _keyword.split('\n')
    _keyword = ' '.join(_keyword)
    _keyword = _keyword.split(' ')
    _keyword = sorted(set(_keyword), key = _keyword.index)   #去掉重复元素
    keyword = sorted(_keyword, lambda x,y: -sortByOrder(text, x, y))
    return keyword

In [32]:
input_path = "/home/jinhanqi/tf_program/summarization_data/gigaword/sumdata/Giga/input.txt"
#input_path = "/home/jinhanqi/tf_program/summarization_data/gigaword/sumdata/train/train.article.txt"
title_path = "/home/jinhanqi/tf_program/summarization_data/gigaword/sumdata/Giga/task1_ref0.txt"

In [33]:
with open(input_path, 'r') as f:
    input = f.readlines()
#input_sample = random.sample(input, 1951)
input_sample = input
keywords_sample = [Get_keywords_and_sort_with_ratio(t, 1.0) for t in input_sample]

### 对数据进行统计

In [34]:
import pandas as pd
import numpy as np

keywords_lens = [len(words) for words in keywords_sample]
length_keywords = pd.DataFrame(keywords_lens, columns=['counts'])

In [35]:
if os.path.exists('cal_keyword_rouge') == False:
    os.mkdir('cal_keyword_rouge')
rouge_ref_dir = 'cal_keyword_rouge/' + "reference"
rouge_dec_dir = 'cal_keyword_rouge/' + "decoded"
if os.path.exists(rouge_ref_dir) == False:
    os.mkdir(rouge_ref_dir)
if os.path.exists(rouge_dec_dir) == False:
    os.mkdir(rouge_dec_dir)

In [36]:
length_keywords.describe()

,counts
count,1951.000000
mean,11.527422
std,4.663898
min,1.000000
25%,8.000000
50%,12.000000
75%,15.000000
max,29.000000


In [39]:
print(np.percentile(length_keywords.counts, 85))

16.0


In [37]:
print(np.percentile(length_keywords.counts, 90))

17.0


In [38]:
print(np.percentile(length_keywords.counts, 95))

19.0


### 计算rouge

In [16]:
with open(title_path, 'r') as f:
    titles = f.readlines()
len(titles)

1951

In [17]:
for ex_index,(words, title) in enumerate(zip(keywords_sample, titles)):
    ref_file = os.path.join(rouge_ref_dir, "%06d_reference.txt" % ex_index)
    decoded_file = os.path.join(rouge_dec_dir, "%06d_decoded.txt" % ex_index)
    with open(ref_file, 'w') as f:
        f.write(title)
    with open(decoded_file, 'w') as f:
        f.write(' '.join(words))
        #f.write(summarizer.summarize(input[ex_index], ratio=1.0))
        #f.write(input[ex_index])

In [18]:
with open(rouge_dec_dir+'/'+"000007_decoded.txt",  'r') as f:
    con = f.read()

In [19]:
con

'israel prepared sunday prime minister yitzhak rabin state funeral world leaders including egyptian heads'

In [20]:
input[7]

"israel prepared sunday for prime minister yitzhak rabin 's state funeral which will be attended by a host of world leaders , including us president bill clinton and the jordanian and egyptian heads of state .\n"

In [21]:
with open(rouge_ref_dir+'/'+"000007_reference.txt",  'r') as f:
    con = f.read()

In [22]:
con

'israel prepares jerusalem state funeral for rabin\n'

In [23]:
%run /home/jinhanqi/program/backup/cal_rouge/rouge.py cal_keyword_rouge

2018-07-22 04:04:51,781 [MainThread  ] [INFO ]  Writing summaries.
2018-07-22 04:04:51,786 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpeSi3JF/system and model files to /tmp/tmpeSi3JF/model.
2018-07-22 04:04:51,789 [MainThread  ] [INFO ]  Processing files in cal_keyword_rouge/decoded.
2018-07-22 04:04:51,819 [MainThread  ] [INFO ]  Processing 000545_decoded.txt.
2018-07-22 04:04:51,821 [MainThread  ] [INFO ]  Processing 001193_decoded.txt.
2018-07-22 04:04:51,823 [MainThread  ] [INFO ]  Processing 000665_decoded.txt.
2018-07-22 04:04:51,825 [MainThread  ] [INFO ]  Processing 001882_decoded.txt.
2018-07-22 04:04:51,827 [MainThread  ] [INFO ]  Processing 000914_decoded.txt.
2018-07-22 04:04:51,828 [MainThread  ] [INFO ]  Processing 000311_decoded.txt.
2018-07-22 04:04:51,830 [MainThread  ] [INFO ]  Processing 000521_decoded.txt.
2018-07-22 04:04:51,832 [MainThread  ] [INFO ]  Processing 001028_decoded.txt.
2018-07-22 04:04:51,834 [MainThread  ] [INFO ]  Pr

2018-07-22 04:04:52,346 [MainThread  ] [INFO ]  Processing 000641_decoded.txt.
2018-07-22 04:04:52,347 [MainThread  ] [INFO ]  Processing 001338_decoded.txt.
2018-07-22 04:04:52,349 [MainThread  ] [INFO ]  Processing 000083_decoded.txt.
2018-07-22 04:04:52,350 [MainThread  ] [INFO ]  Processing 001460_decoded.txt.
2018-07-22 04:04:52,352 [MainThread  ] [INFO ]  Processing 000358_decoded.txt.
2018-07-22 04:04:52,354 [MainThread  ] [INFO ]  Processing 001295_decoded.txt.
2018-07-22 04:04:52,355 [MainThread  ] [INFO ]  Processing 001910_decoded.txt.
2018-07-22 04:04:52,357 [MainThread  ] [INFO ]  Processing 000183_decoded.txt.
2018-07-22 04:04:52,359 [MainThread  ] [INFO ]  Processing 000995_decoded.txt.
2018-07-22 04:04:52,360 [MainThread  ] [INFO ]  Processing 000910_decoded.txt.
2018-07-22 04:04:52,362 [MainThread  ] [INFO ]  Processing 000503_decoded.txt.
2018-07-22 04:04:52,363 [MainThread  ] [INFO ]  Processing 001927_decoded.txt.
2018-07-22 04:04:52,365 [MainThread  ] [INFO ]  Proc

2018-07-22 04:04:52,527 [MainThread  ] [INFO ]  Processing 000704_decoded.txt.
2018-07-22 04:04:52,529 [MainThread  ] [INFO ]  Processing 000333_decoded.txt.
2018-07-22 04:04:52,531 [MainThread  ] [INFO ]  Processing 000934_decoded.txt.
2018-07-22 04:04:52,533 [MainThread  ] [INFO ]  Processing 001831_decoded.txt.
2018-07-22 04:04:52,535 [MainThread  ] [INFO ]  Processing 001665_decoded.txt.
2018-07-22 04:04:52,536 [MainThread  ] [INFO ]  Processing 001542_decoded.txt.
2018-07-22 04:04:52,538 [MainThread  ] [INFO ]  Processing 001600_decoded.txt.
2018-07-22 04:04:52,540 [MainThread  ] [INFO ]  Processing 000297_decoded.txt.
2018-07-22 04:04:52,542 [MainThread  ] [INFO ]  Processing 001375_decoded.txt.
2018-07-22 04:04:52,543 [MainThread  ] [INFO ]  Processing 000299_decoded.txt.
2018-07-22 04:04:52,545 [MainThread  ] [INFO ]  Processing 001238_decoded.txt.
2018-07-22 04:04:52,547 [MainThread  ] [INFO ]  Processing 001435_decoded.txt.
2018-07-22 04:04:52,548 [MainThread  ] [INFO ]  Proc

2018-07-22 04:04:52,715 [MainThread  ] [INFO ]  Processing 001749_decoded.txt.
2018-07-22 04:04:52,717 [MainThread  ] [INFO ]  Processing 000812_decoded.txt.
2018-07-22 04:04:52,718 [MainThread  ] [INFO ]  Processing 001297_decoded.txt.
2018-07-22 04:04:52,720 [MainThread  ] [INFO ]  Processing 001026_decoded.txt.
2018-07-22 04:04:52,722 [MainThread  ] [INFO ]  Processing 000831_decoded.txt.
2018-07-22 04:04:52,724 [MainThread  ] [INFO ]  Processing 001444_decoded.txt.
2018-07-22 04:04:52,726 [MainThread  ] [INFO ]  Processing 001024_decoded.txt.
2018-07-22 04:04:52,727 [MainThread  ] [INFO ]  Processing 001062_decoded.txt.
2018-07-22 04:04:52,729 [MainThread  ] [INFO ]  Processing 000492_decoded.txt.
2018-07-22 04:04:52,731 [MainThread  ] [INFO ]  Processing 000115_decoded.txt.
2018-07-22 04:04:52,733 [MainThread  ] [INFO ]  Processing 000392_decoded.txt.
2018-07-22 04:04:52,735 [MainThread  ] [INFO ]  Processing 000501_decoded.txt.
2018-07-22 04:04:52,737 [MainThread  ] [INFO ]  Proc

2018-07-22 04:04:52,904 [MainThread  ] [INFO ]  Processing 001402_decoded.txt.
2018-07-22 04:04:52,906 [MainThread  ] [INFO ]  Processing 000516_decoded.txt.
2018-07-22 04:04:52,908 [MainThread  ] [INFO ]  Processing 001629_decoded.txt.
2018-07-22 04:04:52,910 [MainThread  ] [INFO ]  Processing 001519_decoded.txt.
2018-07-22 04:04:52,911 [MainThread  ] [INFO ]  Processing 000095_decoded.txt.
2018-07-22 04:04:52,913 [MainThread  ] [INFO ]  Processing 001612_decoded.txt.
2018-07-22 04:04:52,915 [MainThread  ] [INFO ]  Processing 000600_decoded.txt.
2018-07-22 04:04:52,917 [MainThread  ] [INFO ]  Processing 001149_decoded.txt.
2018-07-22 04:04:52,919 [MainThread  ] [INFO ]  Processing 001465_decoded.txt.
2018-07-22 04:04:52,921 [MainThread  ] [INFO ]  Processing 000450_decoded.txt.
2018-07-22 04:04:52,923 [MainThread  ] [INFO ]  Processing 000444_decoded.txt.
2018-07-22 04:04:52,924 [MainThread  ] [INFO ]  Processing 000721_decoded.txt.
2018-07-22 04:04:52,926 [MainThread  ] [INFO ]  Proc

...

2018-07-22 04:04:58,303 [MainThread  ] [INFO ]  Processing 000620_reference.txt.
2018-07-22 04:04:58,305 [MainThread  ] [INFO ]  Processing 000175_reference.txt.
2018-07-22 04:04:58,306 [MainThread  ] [INFO ]  Processing 001282_reference.txt.
2018-07-22 04:04:58,308 [MainThread  ] [INFO ]  Processing 000547_reference.txt.
2018-07-22 04:04:58,310 [MainThread  ] [INFO ]  Processing 000459_reference.txt.
2018-07-22 04:04:58,311 [MainThread  ] [INFO ]  Processing 000846_reference.txt.
2018-07-22 04:04:58,313 [MainThread  ] [INFO ]  Processing 001344_reference.txt.
2018-07-22 04:04:58,315 [MainThread  ] [INFO ]  Processing 000958_reference.txt.
2018-07-22 04:04:58,317 [MainThread  ] [INFO ]  Processing 001762_reference.txt.
2018-07-22 04:04:58,319 [MainThread  ] [INFO ]  Processing 000194_reference.txt.
2018-07-22 04:04:58,321 [MainThread  ] [INFO ]  Processing 000187_reference.txt.
2018-07-22 04:04:58,323 [MainThread  ] [INFO ]  Processing 001047_reference.txt.
2018-07-22 04:04:58,325 [Mai

2018-07-22 04:04:58,483 [MainThread  ] [INFO ]  Processing 001855_reference.txt.
2018-07-22 04:04:58,486 [MainThread  ] [INFO ]  Processing 001405_reference.txt.
2018-07-22 04:04:58,488 [MainThread  ] [INFO ]  Processing 000433_reference.txt.
2018-07-22 04:04:58,490 [MainThread  ] [INFO ]  Processing 001763_reference.txt.
2018-07-22 04:04:58,491 [MainThread  ] [INFO ]  Processing 000963_reference.txt.
2018-07-22 04:04:58,493 [MainThread  ] [INFO ]  Processing 001875_reference.txt.
2018-07-22 04:04:58,495 [MainThread  ] [INFO ]  Processing 001212_reference.txt.
2018-07-22 04:04:58,497 [MainThread  ] [INFO ]  Processing 000761_reference.txt.
2018-07-22 04:04:58,499 [MainThread  ] [INFO ]  Processing 001882_reference.txt.
2018-07-22 04:04:58,501 [MainThread  ] [INFO ]  Processing 000238_reference.txt.
2018-07-22 04:04:58,502 [MainThread  ] [INFO ]  Processing 001230_reference.txt.
2018-07-22 04:04:58,504 [MainThread  ] [INFO ]  Processing 001085_reference.txt.
2018-07-22 04:04:58,506 [Mai

2018-07-22 04:04:58,669 [MainThread  ] [INFO ]  Processing 001808_reference.txt.
2018-07-22 04:04:58,670 [MainThread  ] [INFO ]  Processing 000206_reference.txt.
2018-07-22 04:04:58,672 [MainThread  ] [INFO ]  Processing 001576_reference.txt.
2018-07-22 04:04:58,673 [MainThread  ] [INFO ]  Processing 000545_reference.txt.
2018-07-22 04:04:58,675 [MainThread  ] [INFO ]  Processing 000972_reference.txt.
2018-07-22 04:04:58,677 [MainThread  ] [INFO ]  Processing 001029_reference.txt.
2018-07-22 04:04:58,679 [MainThread  ] [INFO ]  Processing 000554_reference.txt.
2018-07-22 04:04:58,680 [MainThread  ] [INFO ]  Processing 001525_reference.txt.
2018-07-22 04:04:58,682 [MainThread  ] [INFO ]  Processing 001516_reference.txt.
2018-07-22 04:04:58,684 [MainThread  ] [INFO ]  Processing 000784_reference.txt.
2018-07-22 04:04:58,686 [MainThread  ] [INFO ]  Processing 000169_reference.txt.
2018-07-22 04:04:58,688 [MainThread  ] [INFO ]  Processing 001147_reference.txt.
2018-07-22 04:04:58,690 [Mai

2018-07-22 04:04:58,852 [MainThread  ] [INFO ]  Processing 000572_reference.txt.
2018-07-22 04:04:58,854 [MainThread  ] [INFO ]  Processing 001639_reference.txt.
2018-07-22 04:04:58,855 [MainThread  ] [INFO ]  Processing 001495_reference.txt.
2018-07-22 04:04:58,857 [MainThread  ] [INFO ]  Processing 000621_reference.txt.
2018-07-22 04:04:58,858 [MainThread  ] [INFO ]  Processing 001795_reference.txt.
2018-07-22 04:04:58,860 [MainThread  ] [INFO ]  Processing 000514_reference.txt.
2018-07-22 04:04:58,862 [MainThread  ] [INFO ]  Processing 001240_reference.txt.
2018-07-22 04:04:58,863 [MainThread  ] [INFO ]  Processing 001732_reference.txt.
2018-07-22 04:04:58,865 [MainThread  ] [INFO ]  Processing 001129_reference.txt.
2018-07-22 04:04:58,867 [MainThread  ] [INFO ]  Processing 000476_reference.txt.
2018-07-22 04:04:58,869 [MainThread  ] [INFO ]  Processing 000438_reference.txt.
2018-07-22 04:04:58,871 [MainThread  ] [INFO ]  Processing 000167_reference.txt.
2018-07-22 04:04:58,872 [Mai

---------------------------------------------
1 ROUGE-1 Average_R: 0.28682 (95%-conf.int. 0.27719 - 0.29736)
1 ROUGE-1 Average_P: 0.20887 (95%-conf.int. 0.20170 - 0.21667)
1 ROUGE-1 Average_F: 0.22871 (95%-conf.int. 0.22156 - 0.23661)
---------------------------------------------
1 ROUGE-2 Average_R: 0.08938 (95%-conf.int. 0.08303 - 0.09607)
1 ROUGE-2 Average_P: 0.06258 (95%-conf.int. 0.05810 - 0.06715)
1 ROUGE-2 Average_F: 0.06869 (95%-conf.int. 0.06395 - 0.07342)
---------------------------------------------
1 ROUGE-3 Average_R: 0.02254 (95%-conf.int. 0.01862 - 0.02673)
1 ROUGE-3 Average_P: 0.01359 (95%-conf.int. 0.01121 - 0.01620)
1 ROUGE-3 Average_F: 0.01549 (95%-conf.int. 0.01300 - 0.01812)
---------------------------------------------
1 ROUGE-4 Average_R: 0.00666 (95%-conf.int. 0.00389 - 0.00998)
1 ROUGE-4 Average_P: 0.00340 (95%-conf.int. 0.00189 - 0.00533)
1 ROUGE-4 Average_F: 0.00379 (95%-conf.int. 0.00228 - 0.00559)
---------------------------------------------
1 ROUGE-L Aver

In [24]:
with open('cal_keyword_rouge/ROUGE_results.txt', 'r') as f:
    con = f.readlines()

In [29]:
con

['\n',
 'ROUGE-1:\n',
 'rouge_1_f_score: 0.2287 with confidence interval (0.2216, 0.2366)\n',
 'rouge_1_recall: 0.2868 with confidence interval (0.2772, 0.2974)\n',
 'rouge_1_precision: 0.2089 with confidence interval (0.2017, 0.2167)\n',
 '\n',
 'ROUGE-2:\n',
 'rouge_2_f_score: 0.0687 with confidence interval (0.0640, 0.0734)\n',
 'rouge_2_recall: 0.0894 with confidence interval (0.0830, 0.0961)\n',
 'rouge_2_precision: 0.0626 with confidence interval (0.0581, 0.0672)\n',
 '\n',
 'ROUGE-l:\n',
 'rouge_l_f_score: 0.2034 with confidence interval (0.1965, 0.2109)\n',
 'rouge_l_recall: 0.2544 with confidence interval (0.2456, 0.2645)\n',
 'rouge_l_precision: 0.1865 with confidence interval (0.1797, 0.1940)\n']